In [13]:
import pandas as pd
import h2o
import utm
import numpy as np
from utils.to_latlon import to_latlon
from utils.preprocessing import preprocess
from utils.postprocessing import postprocess_data

In [14]:
# Filepaths to input data
pred_data_path = r'C:\Users\John\SynologyDrive\JOBS\~2019\QEJ19152\DATA ANALYSIS\MOVEMENT_ECOLOGY\MASTER_DATA\RTdatMaster20230625.csv'
radio_tower_xy_path = 'Example_data\Input\Radio_tower_locations\RTEastNorth_2groups.xlsx'

# Filepaths to trained models
model_save_path = 'Example_data\Output\Trained_models\MovEcolPaper'

# Variable parameters
freq = '3min' # Frequency of data
data_type = None

routine = 'prediction'
dimensions = ['xOffset', 'yOffset']
predictors = ['ant1_mean', 'ant2_mean', 'ant3_mean', 'ant4_mean', 'ant1_count', 'ant2_count', 'ant3_count', 'ant4_count', 'ant1_std', 'ant2_std', 'ant3_std', 'ant4_std', 'mean_std', 'total_count']

In [15]:
# Get prediction data
pred_data = pd.read_csv(pred_data_path)
pred_data['DateAndTime'] = pd.to_datetime(pred_data['DateAndTime'])

# Get tower locations
tower_locs = pd.read_excel(radio_tower_xy_path)

In [37]:
# Preprocess the unlabelled data
pred_data_preproc, predictors_predict = preprocess(pred_data, freq, routine)

# Create a dictionary of the coordinates of the towers
offset_dict = tower_locs.set_index('TowerID').to_dict()
tower_g = offset_dict['tower_group']

# Add the model group
pred_data_preproc['tower_group'] = pred_data_preproc['TowerID'].map(tower_g).fillna(0)

up to preprocess
created DateTime
down routine prediction route
prediction route subprocess complete


In [38]:
tower_groups = tower_locs['tower_group'].unique()

# Initialise h2o
h2o.init(nthreads = 2)

# Make predictions for each tower group and dimension
for tower_group in tower_groups:
    for dimension in dimensions:
        # Load the trained model
        try:
            if dimension == 'xOffset':
                model = h2o.load_model(f"{model_save_path}\{dimension}_group_{tower_group}_model")
                
            elif dimension == 'yOffset':
                model = h2o.load_model(f"{model_save_path}\{dimension}_group_{tower_group}_model")
            else:
                raise ValueError(f"Error loading the model: {dimension}")
        except Exception as e:
            print(f"Error loading model for dimension '{dimension}': {str(e)}")
    
        # Make predictions on the test data
        data_input = pred_data_preproc[pred_data_preproc['tower_group'] == tower_group]
        unlabelled_data = h2o.H2OFrame(data_input)
        preds = model.predict(unlabelled_data)

        # Save predictions to a new column in the test dataframe
        pred_column_name = f"{dimension}_pred"
        pred_data_preproc.loc[pred_data_preproc['tower_group'] == tower_group, pred_column_name] = preds.as_data_frame().values
        

# Stop h2o
h2o.cluster().shutdown()

Checking whether there is an H2O instance running at http://localhost:54321. connected.
Please download and install the latest version from: https://h2o-release.s3.amazonaws.com/h2o/latest_stable.html


H2O_cluster_uptime:,18 mins 51 secs
H2O_cluster_timezone:,Australia/Sydney
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,7 months and 6 days
H2O_cluster_name:,H2O_from_python_John_2mj5c0
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.437 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
H2O session _sid_8ce1 closed.


In [39]:
predictions = postprocess_data(pred_data_preproc, tower_locs)

In [40]:
predictions[['latitude_pred', 'longitude_pred']] = predictions.apply(lambda row: pd.Series(to_latlon(row['easting_pred'], row['northing_pred'], row['zone_number'], row['zone_letter'])), axis=1)


In [41]:
predictions.to_excel("predictions.xlsx", index=False)